In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import joblib

In [3]:
from vincentvanbot.data import create_flat_images_db, get_data_locally, flat_images_db_download
from vincentvanbot.params import IMAGES_PATH, FLAT_IMAGES_DB_PATH_ROOT, BUCKET_NAME, BUCKET_FLAT_IMAGES_DB_FOLDER
from vincentvanbot.data import get_data_locally, flat_images_db_upload, download_images_locally
from tqdm import tqdm
import os

In [291]:
path = os.path.join('..','..','raw_data','catalog.csv')
df = pd.read_csv(path, encoding= 'unicode_escape') # take care of non-ASCII characters such as 'ö'

df = df[df['FORM'] == 'painting'].head(nrows)
df.head()

,AUTHOR,BORN-DIED,TITLE,DATE,TECHNIQUE,LOCATION,URL,FORM,TYPE,SCHOOL,TIMEFRAME
0,"AACHEN, Hans von","(b. 1552, Köln, d. 1615, Praha)",Venus and Adonis,1574-88,"Oil on canvas, 68 x 95 cm","Fogg Art Museum, Harvard University, Cambridge",https://www.wga.hu/html/a/aachen/adonis.html,painting,mythological,German,1601-1650
1,"AACHEN, Hans von","(b. 1552, Köln, d. 1615, Praha)",Allegory,1598,"Oil on copper, 56 x 47 cm","Alte Pinakothek, Munich",https://www.wga.hu/html/a/aachen/allegory.html,painting,mythological,German,1601-1650
2,"AACHEN, Hans von","(b. 1552, Köln, d. 1615, Praha)","Allegory of Peace, Art and Abundance",1602,"Oil on canvas, 197 x 142 cm","The Hermitage, St. Petersburg",https://www.wga.hu/html/a/aachen/allegorz.html,painting,mythological,German,1601-1650
3,"AACHEN, Hans von","(b. 1552, Köln, d. 1615, Praha)","Jupiter, Antiope and Cupid",1595-98,"Oil on copper, 31 x 21 cm","Kunsthistorisches Museum, Vienna",https://www.wga.hu/html/a/aachen/antiope.html,painting,mythological,German,1601-1650
4,"AACHEN, Hans von","(b. 1552, Köln, d. 1615, Praha)","Pallas Athena, Venus and Juno",1593,"Oil on canvas, 54 x 67 cm","Museum of Fine Arts, Boston",https://www.wga.hu/html/a/aachen/athena.html,painting,mythological,German,1601-1650


In [298]:
df['TECHNIQUE'] = df['TECHNIQUE'].str.replace('\d+', '')
df['TECHNIQUE'] = df['TECHNIQUE'].str.replace(',', '')
df['TECHNIQUE'] = df['TECHNIQUE'].str.replace('cm', '')
df['TECHNIQUE'] = df['TECHNIQUE'].str.replace('x', '')
#df
#s.str.strip('123.!? \n\t')

#df['TECHNIQUE'] = df['TECHNIQUE'].str.strip('123.!? \n\t')
df.tail()
df['TECHNIQUE'].value_counts()

<ipython-input-298-682f78725c70>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['TECHNIQUE'] = df['TECHNIQUE'].str.replace('\d+', '')


Oil on canvas                                    12050
Fresco                                            3751
Oil on panel                                      2691
Oil on canvas                                     2201
Oil on wood                                       1264
                                                 ...  
Gouache an cardboard                                 1
Oil on wood transferred to aluminum panel            1
Oil on canvas over panel                             1
Oil on canvas in gilded frame                        1
Oil on canvas diameter                               1
Name: TECHNIQUE, Length: 1447, dtype: int64

In [264]:
#X = np.array(df_transformed)

In [ ]:
#knn_model = NearestNeighbors(n_neighbors=20).fit(df_transformed)

In [ ]:
df = get_data_locally()
df.shape

In [ ]:
df.iloc[1] 

In [ ]:
df_col_encoded = get_col_transformed(transformer, df)
df_col_encoded

## Recommender System to KNN 

### from title test 

In [148]:
from vincentvanbot.fromtitle.data_rec import create_joined_img_df, create_flat_images_db, get_data_locally, joined_images_db_upload, joined_images_db_download
from vincentvanbot.fromtitle.predict_rec import get_closest_images_indexes, transform_user_input, get_info_from_index
from vincentvanbot.fromtitle.train_rec import train_model, save_model_to_cloud
from vincentvanbot.preprocessing import preprocess_image, build_pipe_for_categorical

In [262]:
nrows=32_008
df = get_data_locally(nrows=nrows)
img_db = create_flat_images_db(size=nrows, path=IMAGES_PATH, dim=(36,42))
download_images_locally(df)
#create_joined_img_df(build_pipe_for_categorical, img_db, size=nrows)


  2%|▌                             | 415/21993 [00:12<10:33, 34.06it/s]


KeyboardInterrupt: 

In [248]:
#joined_images_db_upload(size=nrows, rm=True)
joined_img_df = joined_images_db_download(size=nrows, source='gcp')
print(img_df.shape)

ConnectionError: ('Connection aborted.', timeout('The write operation timed out'))

In [251]:
from vincentvanbot.fromtitle.data_rec import joined_images_db_download
train_df = joined_images_db_download(size=9000, source='gcp', rm=True)
train_model(train_df)
save_model_to_cloud(rm=True)

ConnectionError: HTTPSConnectionPool(host='storage.googleapis.com', port=443): Read timed out.

In [192]:
user_idx = transform_user_input('art', df, case=False)
user_input_transformed = create_joined_img_df(build_pipe_for_categorical, img_db, size=32008).iloc[[user_idx]]
indexes = get_closest_images_indexes(user_idx)
urls = get_info_from_index(indexes)
print(urls)

12830

In [ ]:
def search_title(regex: str, df, case=False):
    """User input is title of image as a string. Search title column of original df, return respective row(s) with any matches."""
    matched_row = df[df['TITLE'].str.contains(regex, regex=True, case=case, na=False)][:1]
    return matched_row

search_title("art", df, case=False)  

In [ ]:
def download_and_preprocess_user_img():
    """Get local path (download), correlating index and apply preprocessing function. Returns flat version of image vector."""
    user_df = search_title(regex: str, df, case=False)
    download_single_image(user_df)
    path = "raw_data/images/..."
    img = load_img(path,target_size=(100,100), interpolation='nearest')
    user_input_transformed = preprocess_image(img, dim=(100,100))
    return user_input_transformed


In [ ]:
# input transformed version of user input to trained KNN
# catalog csv get url - download image locally or request library 


### PCA - check visually with different components

In [ ]:
lfw_path = "/Users/juliaschmitt/code/JulSchmitt/vincentvanbot/raw_data/images"

images = []
for root, dirnames, filenames in os.walk(lfw_path):
    for filename in fnmatch.filter(filenames, '*.jpg'):
        images.append(os.path.join(root, filename))

n = len(images)
print("loaded %d images" % n)

In [ ]:
# random selection of images

img = [Image.open(images[int(n*random.random())]) for i in range(8)]
#img[1] = np.concatenate([np.array(im.getdata()).reshape((im.size[0], im.size[1], 3)) for im in img ], axis=1)
#plt.figure(figsize=(16,2))
#plt.imshow(Image.fromarray(img.astype('uint8')))


#concat_image = np.concatenate([np.asarray(t) for t in img], axis=1)

#img = [[X[idx].reshape((w,h,3))] for idx in np.random.randint(n, size=8)]
#img = np.concatenate([np.asarray(t) for t in img], axis=1)
#plt.figure(figsize=(16,8))
#plt.imshow(Image.fromarray(img_all.astype('uint8')))

#img = img.resize((int(img.width * 100 / img.height), 100))

In [ ]:
n_components = 50
pca = PCA(n_components=n_components, svd_solver='randomized', whiten=True)
pca.fit(X)

In [ ]:
Xp = pca.transform(df)
Xp.shape

In [ ]:
Xt = pca.inverse_transform(Xp)
Xt = np.clip(Xt, 0, 255)

In [ ]:
w = h = 100

In [ ]:
test_1 = X[1].reshape((w,h,3)) #for idx in np.random.randint(n, size=8)][1]
test_2 = Xt[1].reshape((w,h,3)) #for idx in np.random.randint(n, size=8)][1]
stack_img = np.hstack((test_1, test_2))
plt.figure(figsize=(32,4))
plt.imshow(stack_img)


In [ ]:
test_x = [X[idx].reshape((w,h,3)) for idx in np.random.randint(n, size=8)][1]
plt.imshow(test_x)

In [ ]:
test_xt = [Xt[idx].reshape((w,h,3)) for idx in np.random.randint(n, size=8)][1]
plt.imshow(test_xt)

In [ ]:
npca = [100, 50, 10, 2, 1]
Xt = []
for p in npca:
    print("computing PCA for %d components"%p)
    pca = PCA(n_components=p, svd_solver='randomized', whiten=True)
    pca.fit(X)
    Xp = pca.transform(X)
    Xt.append(np.clip(pca.inverse_transform(Xp), 0, 255))

In [ ]:
idxs = np.random.randint(n, size=3)
for i,idx in enumerate(idxs):
    img_orig = X[idx].reshape((w,h,3))
    img_reconstructed = np.concatenate([Xt[p][idx].reshape((w,h,3)) for p in range(len(npca))], axis=1)
    plt.figure(figsize=(2,2))
    plt.imshow(Image.fromarray(img_orig.astype('uint8')))
    plt.figure(figsize=(16,2))
    plt.imshow(Image.fromarray(img_reconstructed.astype('uint8')))


In [ ]:
img_pairs = [[X[idx].reshape((w,h,3)), Xt[idx].reshape((w,h,3))] for idx in np.random.randint(n, size=8)]
img_all = np.concatenate([np.concatenate([p[0], p[1]]) for p in img_pairs], axis=1)
plt.figure(figsize=(32,8))
plt.imshow(Image.fromarray(img_all.astype('uint8')))
